# Robot Simulation with PyCRAM: Object Manipulation and Movement
In this notebook, we will walk through a complete example of setting up a robot environment, creating objects, and moving the robot to detect and interact with those objects using **PyCRAM**. We will explain the code step by step and ensure you understand how each function works in the simulation context.



## Step 1: Initialization

We start by importing the necessary libraries to set up the robot simulation. Each library serves a specific role in controlling the robot, creating the world, and interacting with objects:
TFBroadcaster: This is responsible for managing transformations in the environment, enabling the robot to understand spatial relations.
VizMarkerPublisher: This is used to visualize markers that help track the robot and object locations within the simulated world.
BulletWorld: A class from the PyCRAM framework that helps to create and manage the simulation environment.
ActionDesignator, LocationDesignator, ObjectDesignator: These represent abstract designators for robot actions, locations, and objects respectively, helping to describe tasks and identify targets.
Pose: Represents the position and orientation of objects or robots in the simulated world.
SimulatedRobot: This enables running the robot in a simulated mode for testing without requiring a physical robot.

In [ ]:
from pycram.ros.tf_broadcaster import TFBroadcaster
from pycram.ros.viz_marker_publisher import VizMarkerPublisher, AxisMarkerPublisher
from pycram.worlds.bullet_world import BulletWorld
from pycram.designators.action_designator import *
from pycram.designators.location_designator import *
from pycram.designators.object_designator import *
from pycram.datastructures.enums import ObjectType, WorldMode, TorsoState
from pycram.datastructures.pose import Pose
from pycram.process_module import simulated_robot, with_simulated_robot
from pycram.object_descriptors.urdf import ObjectDescription
from pycram.world_concepts.world_object import Object
from pycram.datastructures.dataclasses import Color
from pycram.designators.object_designator import BelieveObject

extension = ObjectDescription.get_file_extension()

## Step 2: Setting Up the Simulation World
In this step, we set up the simulation environment using `BulletWorld`. We are using the `DIRECT` world mode, which means the simulation runs without a GUI (graphical interface), making it faster and more suitable for headless operation. We also initialize a visualization marker publisher to track objects and visualize movements in the simulation.

- `BulletWorld(WorldMode.DIRECT)`: Creates a simulation world in headless mode (no visual interface).
- `VizMarkerPublisher()`: Visualizes the movements and positions of objects and robots in the simulation world.
TODO: rapid fire simulation robot is teleporting

In [ ]:
world = BulletWorld(WorldMode.DIRECT)
viz = VizMarkerPublisher()

## Step 3: Adding the Robot to the World
Now, we add the robot into the simulation. In this case, we use a PR2 robot model. We define the robot’s type and position in the world using a pose. The pose is a tuple of (x, y, z) coordinates that specify the robot's starting position.

- `Object(robot_name, ObjectType.ROBOT, ...)`: Defines the robot as an object in the world.
- `Pose([1, 2, 0])`: Sets the robot's initial position at coordinates (1, 2, 0).

In [ ]:
pr2 = Object("pr2", ObjectType.ROBOT, "pr2.urdf")     
apartment = Object('apartment', ObjectType.ENVIRONMENT, f'apartment-small{extension}')
milk = Object("milk", ObjectType.MILK, "milk.stl", pose=Pose([1.3, 1, 0.9]))

## Step 4: Understanding Action Designator (NavigateAction)
Action Designators are high-level descriptions of actions which the robot should execute.

Action Designators are created from an Action Designator Description, which describes the type of action as well as the parameter for this action. Parameter are given as a list of possible parameters. For example, if you want to describe the robot moving to a table you would need a NavigateAction and a list of poses that are near the table. The Action Designator Description will then pick one of the poses and return a performable Action Designator which contains the picked pose. A Navigation would look like this:
    
 ```python
 NavigateAction(target_locations=[Pose([1.7, 2, 0])]).resolve().perform()
 ```

To move the robot we need to create a description and resolve it to an actual Designator. The description of navigation only needs a list of possible poses.

In [ ]:
from pycram.designators.action_designator import NavigateAction
from pycram.datastructures.pose import Pose

pose = Pose([1, 0, 0], [0, 0, 0, 1])

# This is the Designator Description
navigate_description = NavigateAction(target_locations=[pose])

# This is the performable Designator
navigate_designator = navigate_description.resolve()    

What we now did was: 
  - create the pose where we want to move the robot
  - create a description describing a navigation with a list of possible poses (in this case the list contains only one pose) and
  - create an action designator from the description

The action designator contains the pose picked from the list of possible poses and can then be performed.


Every designator that is performed needs to be in an environment that specifies where to perform the designator: Either on the real robot or the simulated one. 
This environment is called simulated_robot. 
If we want to perform actions on the real robot we would use real_robot instead.

There are also decorators which do the same thing but for whole methods, they are called with_real_robot and with_simulated_robot.

In [ ]:
from pycram.process_module import simulated_robot

with simulated_robot:
    navigate_designator.perform()

## Step 5: Let the robot look at the object

The LookAtAction is used to make the robot look at a specific

In [ ]:
from pycram.designators.action_designator import LookAtAction
from pycram.process_module import simulated_robot
from pycram.datastructures.pose import Pose

target_location = Pose([1, 0, 0.5], [0, 0, 0, 1])
with simulated_robot:
    LookAtAction(targets=[target_location]).resolve().perform()

## Step 6: Detect the object

The DetectAction is used to detect objects in the field of vision (FOV) of the robot. The detect designator will return a resolved instance of an ObjectDesignatorDescription.
```python
obj_desig = DetectAction(milk_desig).resolve().perform()
```

## Step 7: Robot Movement and Object Detection
In this step, we define how the robot moves through the environment and detects objects. The robot is instructed to move to a target location, look at a specific object (in this case, a milk carton), and detect it. These actions are handled by designators like NavigateAction for moving, LookAtAction for directing its gaze, and DetectAction for identifying objects.

- NavigateAction: Moves the robot to the specified coordinates.

- LookAtAction: Rotates the robot to look at the given object or location.

- DetectAction: Enables the robot to detect and recognize an object based on its type.## Believe Object
Todo: Make clear that people understand the difference in believe object and object

This object designator is used to describe objects that are located in the BulletWorld. So objects that are in the
belief state, hence the name. In the future when there is a perception interface, there will be a ```RealObject```
description which will be used to describe objects in the real world.

Since {meth}`~pycram.designators.object_designator.BelieveObject` describes Objects in the BulletWorld we create a few.

```python
kitchen = Object("kitchen", ObjectType.ENVIRONMENT, "kitchen.urdf")
milk = Object("milk", ObjectType.MILK, "milk.stl", pose=Pose([1.3, 1, 0.9]))
cereal = Object("froot_loops", ObjectType.BREAKFAST_CEREAL, "breakfast_cereal.stl", pose=Pose([1.3, 0.9, 0.95]))
spoon = Object("spoon", ObjectType.SPOON, "spoon.stl", pose=Pose([1.3, 1.1, 0.87]))
```

Now that we have objects we can create an object designator to describe them. To start off we want an object designator
only describing the milk. Since all objects have unique names we can create an object designator using a list with only
the name of the object.

```python
from pycram.designators.object_designator import BelieveObject

object_description = BelieveObject(names=["milk"])

print(object_description.resolve())
```

You can also use the type to describe objects, so now we want to have an object designator that describes every food in
the world.

```python
from pycram.designators.object_designator import BelieveObject

object_description = BelieveObject(types=[ObjectType.MILK, ObjectType.BREAKFAST_CEREAL])

print(object_description.resolve())
```
So now you can use the robot_desig, apartment_desig and the milk_desig to further programm!

In [ ]:
milk_desig = BelieveObject(names=["milk"])

## add your code here

## Solution

<details>

<summary>Click here to get the solution</summary>

```python
from pycram.designators.action_designator import DetectAction, LookAtAction, ParkArmsAction, NavigateAction
from pycram.designators.object_designator import BelieveObject
from pycram.datastructures.enums import Arms
from pycram.process_module import simulated_robot
from pycram.datastructures.pose import Pose

milk_desig = BelieveObject(names=["milk"])

with simulated_robot:
    ParkArmsAction([Arms.BOTH]).resolve().perform()

    NavigateAction([Pose([0, 1, 0], [0, 0, 0, 1])]).resolve().perform()

    LookAtAction(targets=[milk_desig.resolve().pose]).resolve().perform()

    obj_desig = DetectAction(milk_desig).resolve().perform()

    print(obj_desig)
```
</details>

## Step 8: Understanding the object designator
Object designators are used to describe objects located in the BulletWorld or the real environment and then resolve them during runtime to concrete objects.

Object designators are different from the Object class in bullet_world.py in the way that they just describe an object and do not create objects or provide methods to manipulate them. Nevertheless, object designators contain a reference to the BulletWorld object.

An Object designator takes two parameters, of which at least one has to be provided. These parameters are:

A list of names
A list of types
Object Designators work similar to Location designators, they get constrains describing a set of objects and when resolved return a specific instance.
The robot recognizes the milk carton as an object with the following description:

In [ ]:
ObjectDesignatorDescription.Object(name="milk", obj_type=ObjectType.MILK, 
    world_object=Object(id=3, world=<BulletWorld>, name="milk", obj_type=ObjectType.MILK, 
    color=Color(R=1, G=1, B=1, A=1), 
    pose=Pose(position=[1.3, 1.0, 0.9], orientation=[0.0, 0.0, 0.0, 1.0])))

1. ID and Saved States
ID: 3: The object has an identifier (ID) of 3, which allows it to be uniquely recognized in the world.
Saved States: {}: No specific states are saved for this object yet, but this can be useful if the robot needs to remember object states (like previous positions).
2. World Information
World: The object exists within the simulation's BulletWorld, the environment where all interactions occur.
Name: The object is named "milk", so the robot knows this is the milk carton.
Object Type: The object is of type ObjectType.MILK, telling the robot it's dealing with a milk carton.
3. Physical Properties
Color: The milk carton is white, represented as Color(R=1, G=1, B=1, A=1) using RGBA values (Red, Green, Blue, Alpha). This helps the robot visually identify it.
Pose:
Position: The milk is located at coordinates x=1.3, y=1.0, and z=0.9, defining its place in the 3D world.
Orientation: The milk has no rotation, with its orientation set to [0.0, 0.0, 0.0, 1.0]. These values define how the object is aligned in the world.
4. URDF Model
URDF Path: The milk is modeled using a file located at /resources/cached/milk.urdf. URDF files contain the visual and physical properties of objects, allowing the simulation to render the object correctly.
5. Frames and Transformations
Transformation Frame: The frame used for calculating transformations is set to "milk". This frame helps the robot understand where the milk is relative to other objects or itself.
Pose Information: The object’s pose can be retrieved using methods like get_pose(), allowing the robot to track its current location.
6. Cache Manager and Attachments
Cache Manager: The robot uses caching to improve performance by managing states efficiently.
Attachments: There are no other objects attached to the milk. This would be relevant if the object were, for example, inside a drawer or held by a robot arm.

One of the great features of PyCRAM is its flexibility in handling both simulated and real environments seamlessly. Regardless of whether we interact with objects in a simulation or the real world, the designator remains the same! In simulation, object detection doesn't rely on a real camera—instead, the system simulates the detection process. However, when using the same detection action on a real robot, PyCRAM integrates with tools like Robokudo to recognize objects in the physical environment. This capability allows us to easily switch between simulation and reality using the same code, which makes development highly efficient. That said, it's important to remember that the real world is much more complex than the simulation, which is only a simplified model. Nonetheless, the ability to use the same code for both simulated and real robots significantly simplifies transitioning between the two environments.

## Step 9: Moving the milk Carton inside the fridge

In [ ]:
milk = Object("milk", ObjectType.MILK, "milk.stl", pose=Pose([1.3, 1, 0.9]))
pycram.plan_failures.PerceptionObjectNotFound: Could not find an object with the type ObjectType.MILK in the FOV of the robot
